In [1]:
import sys
sys.path.append('../src/mane/prototype/')
import numpy as np
import graph as g
import pickle as p

from sklearn.preprocessing import normalize, scale
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [2]:
def lg(exp_id, graph_name, index=[0], norm=False, split=0.5, max_iter=100, C=1e9, ic=500):
    weightfile = '../src/mane/prototype/embeddings/' + exp_id + '.weights'
    graphfile = '../src/mane/data/' + graph_name
    with open(weightfile, 'rb') as f:
        w = p.load(f)
    graph = g.graph_from_pickle(graphfile+'.graph', graphfile+'.community')
    emb = None
    if index is None:
        emb = w
    else:
        for i in index:
            if emb is None:
                emb = w[i]
            else:
                emb += w[i]
        emb /= len(index)
    if norm:
        emb = normalize(emb)
    xids, y_train = graph.gen_training_community(split)
    X = [emb[i] for i in xids]
    predictor = LogisticRegression(C=C, max_iter=max_iter, 
                                   n_jobs=-1, intercept_scaling=ic).fit(X, y_train)
    y_true = [graph._communities[i] for i in graph.nodes()]
    y_pred = [predictor.predict(emb[i].reshape(1,-1))[0] for i in graph.nodes()]
    print('Experiment ', exp_id, ' ', graph_name)
    print('f1_macro (emb): ', f1_score(y_true, y_pred, average='macro'))
    print('f1_micro (emb): ', f1_score(y_true, y_pred, average='micro'))
    return y_true, y_pred, emb, predictor, xids, y_train, graph

In [5]:
# Evaluation excluding training data 
def lg_blind(exp_id, graph_name, index=[0], norm=False, split=0.5, max_iter=100, C=1e9, ic=500):
    weightfile = '../src/mane/prototype/embeddings/' + exp_id + '.weights'
    graphfile = '../src/mane/data/' + graph_name
    with open(weightfile, 'rb') as f:
        w = p.load(f)
    graph = g.graph_from_pickle(graphfile+'.graph', graphfile+'.community')
    emb = None
    if index is None:
        emb = w
    else:
        for i in index:
            if emb is None:
                emb = w[i]
            else:
                emb += w[i]
        emb /= len(index)
    if norm:
        emb = normalize(emb)
    xids, y_train = graph.gen_training_community(split)
    X = [emb[i] for i in xids]
    predictor = LogisticRegression(C=C, max_iter=max_iter, 
                                   n_jobs=-1, intercept_scaling=ic).fit(X, y_train)
    eval_list = [i for i in graph.nodes() if i not in xids]
    y_true = [graph._communities[i] for i in eval_list]
    y_pred = [predictor.predict(emb[i].reshape(1,-1))[0] for i in eval_list]
    print('Experiment ', exp_id, ' ', graph_name)
    print('f1_macro (emb): ', f1_score(y_true, y_pred, average='macro'))
    print('f1_micro (emb): ', f1_score(y_true, y_pred, average='micro'))
    return y_true, y_pred, emb, predictor, xids, y_train, graph

In [3]:
for _ in range(10):
    lg('BC3027', 'blogcatalog3')

Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.219340810808
f1_micro (emb):  0.255236617533
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.219550623436
f1_micro (emb):  0.256594259116
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.217514126395
f1_micro (emb):  0.251745539178
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.205920937698
f1_micro (emb):  0.248545384019
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.212722238944
f1_micro (emb):  0.251745539178
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.212169575093
f1_micro (emb):  0.250193948798
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.209820736239
f1_micro (emb):  0.251260667184
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.219841558458
f1_micro (emb):  0.250969743988
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.222914009055
f1_micro (emb):  0.252715283165
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.21712996816
f1_micro (emb):  0.249806051202


In [4]:
for _ in range(10):
    lg('BC3027', 'blogcatalog3', max_iter=1000, C=1e15, ic=210)

Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.220095321555
f1_micro (emb):  0.252133436773
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.210292012732
f1_micro (emb):  0.248836307215
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.213477478626
f1_micro (emb):  0.252618308766
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.216231290275
f1_micro (emb):  0.249515128006
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.209187291432
f1_micro (emb):  0.250193948798
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.212264060928
f1_micro (emb):  0.250484871994
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.210344662384
f1_micro (emb):  0.251745539178
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.210834861987
f1_micro (emb):  0.244763382467
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.21620488597
f1_micro (emb):  0.253782001552
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.209572979136
f1_micro (emb):  0.25232738557


In [6]:
# Experiments excluding training data shows extremely bad results
for _ in range(10):
    lg_blind('BC3027', 'blogcatalog3')

Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0931810396325
f1_micro (emb):  0.180534055728


/home/hoangnt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.10314286853
f1_micro (emb):  0.188854489164
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0951006520018
f1_micro (emb):  0.179373065015
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.101985864487
f1_micro (emb):  0.190595975232
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0935649786308
f1_micro (emb):  0.184017027864
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0936123450619
f1_micro (emb):  0.179179566563
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0904955309742
f1_micro (emb):  0.180921052632
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.10350875807
f1_micro (emb):  0.188660990712
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0952439613814
f1_micro (emb):  0.18169504644
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0971542525409
f1_micro (emb):  0.184404024768


In [8]:
# Experiments excluding training data shows extremely bad results
for _ in range(10):
    lg_blind('BC3027', 'blogcatalog3', C=1, ic=1)

Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0952686995995
f1_micro (emb):  0.202786377709


/home/hoangnt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0944912230997
f1_micro (emb):  0.205882352941
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0952417410481
f1_micro (emb):  0.206462848297
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0965639753398
f1_micro (emb):  0.208010835913
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0908257616305
f1_micro (emb):  0.198722910217
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0972160058366
f1_micro (emb):  0.206849845201
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0913322993109
f1_micro (emb):  0.203366873065
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0976298357878
f1_micro (emb):  0.207623839009
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0898440597327
f1_micro (emb):  0.201238390093
Experiment  BC3027   blogcatalog3
f1_macro (emb):  0.0932630011112
f1_micro (emb):  0.195433436533


In [10]:
for _ in range(10):
    lg('BC3028', 'blogcatalog3', max_iter=1000, C=1e15, ic=210)

Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.213999403766
f1_micro (emb):  0.253975950349
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.223045798608
f1_micro (emb):  0.263479441427
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.221812481446
f1_micro (emb):  0.261442979054
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.223815063832
f1_micro (emb):  0.260473235066
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.21985679345
f1_micro (emb):  0.259697439876
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.224638568968
f1_micro (emb):  0.263479441427
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.226129532679
f1_micro (emb):  0.262897595035
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.234221592424
f1_micro (emb):  0.268813033359
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.218258691716
f1_micro (emb):  0.261055081458
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.217104213563
f1_micro (emb):  0.261927851047


In [11]:
for _ in range(10):
    lg_blind('BC3028', 'blogcatalog3', C=1, ic=1)

Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.09448175907
f1_micro (emb):  0.212267801858


/home/hoangnt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.0964194164401
f1_micro (emb):  0.215750773994
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.100618458012
f1_micro (emb):  0.218459752322
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.0948350015475
f1_micro (emb):  0.209945820433
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.101391400206
f1_micro (emb):  0.215557275542
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.100458419123
f1_micro (emb):  0.22020123839
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.10150633818
f1_micro (emb):  0.222716718266
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.0942622292313
f1_micro (emb):  0.210913312693
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.0978910661161
f1_micro (emb):  0.214783281734
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.100047075437
f1_micro (emb):  0.215557275542


In [12]:
for _ in range(10):
    lg('BC3028', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e15, ic=210)

Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.229724495351
f1_micro (emb):  0.270655546936
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.229977626154
f1_micro (emb):  0.271140418929
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.240309790505
f1_micro (emb):  0.272401086113
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.235426010844
f1_micro (emb):  0.274340574088
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.237096200622
f1_micro (emb):  0.27579519007
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.238224967573
f1_micro (emb):  0.272788983708
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.239630380058
f1_micro (emb):  0.273855702095
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.235116407269
f1_micro (emb):  0.271819239721
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.242535597115
f1_micro (emb):  0.275116369279
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.244063926108
f1_micro (emb):  0.276377036462


In [13]:
for _ in range(10):
    lg_blind('BC3028', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e15, ic=210)

Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.101678266365
f1_micro (emb):  0.191176470588


/home/hoangnt/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.102267019126
f1_micro (emb):  0.198529411765
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.106337224615
f1_micro (emb):  0.199690402477
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.107150241596
f1_micro (emb):  0.198722910217
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.110716302277
f1_micro (emb):  0.198722910217
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.110737982459
f1_micro (emb):  0.201044891641
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.0986068793443
f1_micro (emb):  0.18943498452
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.109800777535
f1_micro (emb):  0.202205882353
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.111685568855
f1_micro (emb):  0.204140866873
Experiment  BC3028   blogcatalog3
f1_macro (emb):  0.107677324416
f1_micro (emb):  0.196594427245


In [16]:
for _ in range(10):
    lg('BC3029', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e15, ic=210)

Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.18958969791
f1_micro (emb):  0.196761055081
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.17449966687
f1_micro (emb):  0.191718386346
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.176251036602
f1_micro (emb):  0.194142746315
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.174661689799
f1_micro (emb):  0.192106283941
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.172795906686
f1_micro (emb):  0.195888285493
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.174497886162
f1_micro (emb):  0.193366951125
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.177250439601
f1_micro (emb):  0.197439875873
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.177302173004
f1_micro (emb):  0.19608223429
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.174454829719
f1_micro (emb):  0.193657874321
Experiment  BC3029   blogcatalog3
f1_macro (emb):  0.182991307525
f1_micro (emb):  0.196761055081


In [17]:
for _ in range(10):
    lg('BC3026', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e15, ic=210)

Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.222043426209
f1_micro (emb):  0.257854926299
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.219794769952
f1_micro (emb):  0.257564003103
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.22006756222
f1_micro (emb):  0.257951900698
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.227797875877
f1_micro (emb):  0.265515903801
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.215934186811
f1_micro (emb):  0.256012412723
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.227319015001
f1_micro (emb):  0.263576415826
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.216641149058
f1_micro (emb):  0.258242823894
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.223035082032
f1_micro (emb):  0.261346004655
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.225589080412
f1_micro (emb):  0.260473235066
Experiment  BC3026   blogcatalog3
f1_macro (emb):  0.229426360561
f1_micro (emb):  0.260473235066


In [18]:
for _ in range(10):
    lg('BC3030', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e15, ic=210)

Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.206922506043
f1_micro (emb):  0.238557020946
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.205417751205
f1_micro (emb):  0.243793638479
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.206986702687
f1_micro (emb):  0.239138867339
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.210225076664
f1_micro (emb):  0.239138867339
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.210988978063
f1_micro (emb):  0.236520558573
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.203111361552
f1_micro (emb):  0.238750969744
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.208442704712
f1_micro (emb):  0.237878200155
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.205469059113
f1_micro (emb):  0.236617532971
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.212362464679
f1_micro (emb):  0.242629945694
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.212186946857
f1_micro (emb):  0.240011636928


In [22]:
for _ in range(10):
    lg('BC3030', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e15, ic=100)

Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.203421626021
f1_micro (emb):  0.234484096199
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.205568633593
f1_micro (emb):  0.236520558573
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.211892651072
f1_micro (emb):  0.245442203258
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.19891154967
f1_micro (emb):  0.236811481769
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.202837434806
f1_micro (emb):  0.236326609775
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.205352228515
f1_micro (emb):  0.23904189294
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.205673619484
f1_micro (emb):  0.23826609775
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.210680024029
f1_micro (emb):  0.239138867339
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.207390430664
f1_micro (emb):  0.238460046548
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.206941520282
f1_micro (emb):  0.238944918542


In [23]:
for _ in range(10):
    lg('BC3030', 'blogcatalog3', index=[0,1], norm=True, max_iter=10000, C=1e15, ic=210)

Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.207688328391
f1_micro (emb):  0.23749030256
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.207832811778
f1_micro (emb):  0.238169123351
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.208303269935
f1_micro (emb):  0.239526764934
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.21378127655
f1_micro (emb):  0.242532971296
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.207565866071
f1_micro (emb):  0.237587276959
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.20539546591
f1_micro (emb):  0.23826609775
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.20597198371
f1_micro (emb):  0.24059348332
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.208138322901
f1_micro (emb):  0.241660201707
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.204190860758
f1_micro (emb):  0.240205585725
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.204120956643
f1_micro (emb):  0.236132660978


In [24]:
for _ in range(10):
    lg('BC3003', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e15, ic=210)

Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.233033837672
f1_micro (emb):  0.271528316524
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.230738178138
f1_micro (emb):  0.270267649341
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.227387998884
f1_micro (emb):  0.265321955004
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.233004492462
f1_micro (emb):  0.269588828549
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.239470766465
f1_micro (emb):  0.271528316524
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.241236353361
f1_micro (emb):  0.275407292475
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.222317428962
f1_micro (emb):  0.264255236618
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.234494567131
f1_micro (emb):  0.270170674942
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.237510405322
f1_micro (emb):  0.270849495733
Experiment  BC3003   blogcatalog3
f1_macro (emb):  0.229881649379
f1_micro (emb):  0.270461598138


In [25]:
for _ in range(10):
    lg('BC3030', 'blogcatalog3', index=[0], norm=True, max_iter=10000, C=1e15, ic=210)

Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.203061803854
f1_micro (emb):  0.228859581071
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.202054897974
f1_micro (emb):  0.226435221102
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.20635900728
f1_micro (emb):  0.231380915438
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.204092102046
f1_micro (emb):  0.231574864236
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.20210079459
f1_micro (emb):  0.230605120248
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.207998710109
f1_micro (emb):  0.23205973623
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.194943247732
f1_micro (emb):  0.229150504267
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.205290351251
f1_micro (emb):  0.229150504267
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.208777421792
f1_micro (emb):  0.231380915438
Experiment  BC3030   blogcatalog3
f1_macro (emb):  0.203677020006
f1_micro (emb):  0.229538401862


In [27]:
for _ in range(20):
    lg('BC3031', 'blogcatalog3', index=[0,1], norm=True, max_iter=10000, C=1e15, ic=210)

Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.240672976707
f1_micro (emb):  0.269879751746
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.230497990666
f1_micro (emb):  0.271237393328
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.237929096648
f1_micro (emb):  0.272595034911
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.229928243397
f1_micro (emb):  0.270267649341
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.229259080734
f1_micro (emb):  0.266097750194
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.238409564314
f1_micro (emb):  0.271625290923
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.235458655236
f1_micro (emb):  0.268231186967
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.235367149094
f1_micro (emb):  0.271043444531
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.234103008224
f1_micro (emb):  0.269394879752
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.231770331552
f1_micro (emb):  0.261636927851
Experiment  BC3031  

In [28]:
for _ in range(20):
    lg('BC3031', 'blogcatalog3', index=[0,1], norm=True, max_iter=10000, C=1e5, ic=210)

Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.231828283846
f1_micro (emb):  0.268813033359
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.235546561489
f1_micro (emb):  0.268037238169
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.228151428361
f1_micro (emb):  0.265515903801
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.238741082506
f1_micro (emb):  0.270461598138
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.241483711331
f1_micro (emb):  0.271237393328
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.23574927333
f1_micro (emb):  0.26871605896
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.240854370465
f1_micro (emb):  0.272304111715
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.235189533622
f1_micro (emb):  0.272207137316
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.236245683791
f1_micro (emb):  0.268231186967
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.230918160449
f1_micro (emb):  0.267746314973
Experiment  BC3031   b

In [30]:
for _ in range(20):
    lg('BC3031', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e5, ic=100)

Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.229657949154
f1_micro (emb):  0.270461598138
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.234023826671
f1_micro (emb):  0.268134212568
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.238282761491
f1_micro (emb):  0.270655546936
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.236492433709
f1_micro (emb):  0.269394879752
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.239963130321
f1_micro (emb):  0.273176881303
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.232781827575
f1_micro (emb):  0.269588828549
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.236982080389
f1_micro (emb):  0.267552366175
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.235220744534
f1_micro (emb):  0.269297905353
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.236232695193
f1_micro (emb):  0.269879751746
Experiment  BC3031   blogcatalog3
f1_macro (emb):  0.235287917439
f1_micro (emb):  0.269685802948
Experiment  BC3031  

In [31]:
for _ in range(20):
    lg('BC3032', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e5, ic=100)

Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.240419909214
f1_micro (emb):  0.271722265322
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.230847128314
f1_micro (emb):  0.263479441427
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.225819522635
f1_micro (emb):  0.263479441427
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.228839106084
f1_micro (emb):  0.262024825446
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.23627237559
f1_micro (emb):  0.264740108611
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.229323495818
f1_micro (emb):  0.264740108611
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.228719722846
f1_micro (emb):  0.265903801396
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.237586336544
f1_micro (emb):  0.269200930954
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.234402327581
f1_micro (emb):  0.270170674942
Experiment  BC3032   blogcatalog3
f1_macro (emb):  0.235243716857
f1_micro (emb):  0.262218774244
Experiment  BC3032   

In [32]:
for _ in range(20):
    lg('BC3033', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e5, ic=100)

Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.203276893425
f1_micro (emb):  0.240399534523
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.20522086384
f1_micro (emb):  0.238363072149
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.202127148942
f1_micro (emb):  0.241078355314
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.204913232543
f1_micro (emb):  0.238072148953
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.208032468277
f1_micro (emb):  0.244375484872
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.207987952883
f1_micro (emb):  0.2343871218
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.203115477224
f1_micro (emb):  0.240981380915
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.19910095626
f1_micro (emb):  0.238363072149
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.200956624974
f1_micro (emb):  0.237684251358
Experiment  BC3033   blogcatalog3
f1_macro (emb):  0.199898729464
f1_micro (emb):  0.23671450737
Experiment  BC3033   blog

In [33]:
for _ in range(20):
    lg('BC3034', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e5, ic=100)

Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.210095303911
f1_micro (emb):  0.244375484872
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.201663156906
f1_micro (emb):  0.238072148953
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.197266026583
f1_micro (emb):  0.235453840186
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.208435493364
f1_micro (emb):  0.242048099302
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.208370636839
f1_micro (emb):  0.244181536074
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.211010489639
f1_micro (emb):  0.241078355314
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.199649007801
f1_micro (emb):  0.23671450737
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.20907430234
f1_micro (emb):  0.241854150504
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.205003856236
f1_micro (emb):  0.238557020946
Experiment  BC3034   blogcatalog3
f1_macro (emb):  0.208399485822
f1_micro (emb):  0.242242048099
Experiment  BC3034   b

In [34]:
for _ in range(20):
    lg('BC3035', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e5, ic=100)

Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.22425380906
f1_micro (emb):  0.264255236618
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.236400617887
f1_micro (emb):  0.268522110163
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.232412454974
f1_micro (emb):  0.264449185415
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.226374346116
f1_micro (emb):  0.263479441427
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.232033829998
f1_micro (emb):  0.266776570985
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.22669907025
f1_micro (emb):  0.261055081458
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.239329240135
f1_micro (emb):  0.264158262219
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.237367674002
f1_micro (emb):  0.263091543832
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.224711856757
f1_micro (emb):  0.266097750194
Experiment  BC3035   blogcatalog3
f1_macro (emb):  0.236798135408
f1_micro (emb):  0.266097750194
Experiment  BC3035   b

In [ ]:
for _ in range(20):
    lg('BC3036', 'blogcatalog3', index=[0,1], norm=True, max_iter=1000, C=1e5, ic=100)